In [1]:
import pandas as pd
from models import get_conn
import geopandas as gpd
import googlemaps
import os
from dotenv import load_dotenv
pd.set_option('display.expand_frame_repr', False) # display full data in terminal

load_dotenv()
engine, Session = get_conn()

In [2]:
# Use acutal google maps API instead of just web scraping. Issues: I'm using the 'Title' of each pin, which didn't return an exact match for all locations. I manually edited ~20 locations in the CSV,
# Adding extra keyworks to teh title (often a state), to get the goglemaps api to return the right result. I still have 4 rows that google couldn't match. Since some titles may return multiple
# results this doesn't guarantee an exact match.

api_key = os.getenv('GOOGLE_API_KEY')
gmaps = googlemaps.Client(key=api_key)

def get_lat_long(title):
    # Geocode the place to get coordinates
    geocode_result = gmaps.geocode(title)
    if geocode_result:
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        return lat, lng
    else:
        return None, None


def fetch_all_lat_long(df):
    latitudes = []
    longitudes = []

    for _, location in df.iterrows():
        lat, lng = get_lat_long(location.title)
        latitudes.append(lat)
        longitudes.append(lng)

    df['latitude'] = latitudes
    df['longitude'] = longitudes
    return df

# locations_df_raw = pd.read_csv('csv_files/2023_2024_Van_Trip.csv')
locations_df_raw = pd.read_csv('csv_files/additional_van_stops.csv')
locations_df_raw['notes'] = locations_df_raw['notes'].fillna("2023 2024 Van Trip")
locations_df_raw.drop(columns=['comment', 'url'], inplace=True)

locations_df = fetch_all_lat_long(locations_df_raw)

# There were ~4 locations that I couldnt get googleapi to return results for.
locations_df.dropna(inplace=True)

In [4]:
# save to waypoints: we have now added lat/long/geom column

gdf = gpd.GeoDataFrame(locations_df, geometry=gpd.points_from_xy(locations_df.longitude, locations_df.latitude), crs="EPSG:4326")

with Session() as session:
    gdf.to_postgis(
        name='waypoints',
        con=engine,
        if_exists='append'
    )
    


In [ ]:
# THIS FAILED DUE TO GOOGLE RATE LIMITING!! SKIP TO NEXT CELL. leaving here for posterity
# Googles exported CSV of places doesn't have Lat/Long, just a url for each location. Fetch each url, look at the rediurect URL, which embedds the Lat/Long

# locations_df_raw = pd.read_csv('csv_files/2023_2024_Van_Trip.csv')
# locations_df_raw['notes'] = locations_df['notes'].fillna("2023 2024 Van Trip")

# def populate_lat_long(df):
#     latitudes = []
#     longitudes = []

#     for _, location in df.iterrows():
#         redirected_url = get_redirect_url(location['url'])
#         lat, long = extract_lat_long_from_redirect(redirected_url)
#         latitudes.append(lat)
#         longitudes.append(long)

#     df['latitude'] = latitudes
#     df['longitude'] = longitudes
#     return df

# def get_redirect_url(url):
#     response = requests.get(url, timeout=10)
#     response.raise_for_status()
#     return response.url

# def extract_lat_long_from_redirect(url):
#     try:
#         parsed_url = urlparse(url)
#         # query_params = parse_qs(parsed_url.query)
        
#         # Look for `3d` and `4d` in the URL path
#         path_segments = parsed_url.path.split('/')
#         for segment in path_segments:
#             if '3d' in segment and '4d' in segment:
#                 # Extract latitude and longitude
#                 parts = segment.split('!')
#                 lat = next((p[2:] for p in parts if p.startswith('3d')), None)
#                 lng = next((p[2:] for p in parts if p.startswith('4d')), None)
#                 return lat, lng
#         return None, None
#     except Exception as e:
#         print(f"Error processing URL {url}: {e}")
#         return None, None

# locations_df = populate_lat_long(locations_df[:5])
# locations_df.head()
